In [14]:
import os
from os import environ

import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt

In [15]:
#route = environ['route']
route='LUX-CPH'
downweight_granularity=['FltNum','dtime','Direction','dday','month']

# Data Import

In [16]:
Data=pd.read_csv('~/SQLsnippets/SQLsnippet_'+route+'.csv',sep=',')

In [18]:
Data=Data.drop_duplicates(subset=['extraction_date','calendar_date','flight_number','airport_arrival','airport_depart','dep_time'])

In [20]:
Data=Data.dropna(how='any')
Data.index=range(Data.shape[0])
Data=Data[['airport_depart','airport_arrival','flight_number','dep_time','extraction_date', 'calendar_date',
     'pax_total','group_pax','cabin_capacity']]
Data=Data.rename(columns={'airport_depart': 'SegFrom', 'airport_arrival': 'SegTo', 'flight_number': 'FltNum', 
'dep_time': 'DepTime', 'extraction_date': 'IssueDate', 'calendar_date': 'DepDate',
'pax_total': 'NumPax', 'group_pax': 'Group pax', 'cabin_capacity': 'Cabin Capacity'})

# CHANGE FORMAT

Data['FltNum']=Data['FltNum'].apply(lambda x: str(str(x).split('-')[1]))
Data['NumPax']=Data['NumPax'].apply(lambda x: float(str(x).split(' ')[0]))
Data['Group pax']=Data['Group pax'].apply(lambda x: float(str(x).split(' ')[0]))
Data['Cabin Capacity']=Data['Cabin Capacity'].apply(lambda x: float(str(x).split(' ')[0]))
Data['DepDayMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[0]+'/'+str(x).split('/')[1])
Data['DepMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[1]+'/'+str(x).split('/')[2])

# MAP DESTINATION COMBINATION TO INBOUND OR OUTBOUND

Map_AirportsToDirection=DataFrame([[route.split('-')[0],route.split('-')[1]],
[route.split('-')[1],route.split('-')[0]],['O','I']]).transpose()
Map_AirportsToDirection.columns=['SegFrom','SegTo','Direction']
Data=Data.merge(Map_AirportsToDirection,on=['SegTo','SegFrom'])

### Change format of remaining columns

In [21]:
IssueDate=Data['IssueDate'].unique().tolist()
IssueDate_Map=DataFrame([IssueDate,
[pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y') for x in IssueDate]]).transpose()
IssueDate_Map.columns=['IssueDate','NewFormat']

Data=Data.merge(IssueDate_Map,on=['IssueDate'])
Data=Data[[x for x in Data.columns if x!='IssueDate']]
Data=Data.rename(columns={'NewFormat': 'IssueDate'})

#######################################

DepartureDate=Data['DepDate'].unique().tolist()
DepartureDate_Map=DataFrame([DepartureDate,
[pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y') for x in DepartureDate]]).transpose()
DepartureDate_Map.columns=['DepDate','NewFormat']
DepartureDate_Map['yday']=DepartureDate_Map['NewFormat'].apply(lambda x: x.timetuple().tm_yday)

DepartureDate_Map['dday_num']=DepartureDate_Map['NewFormat'].apply(lambda x: x.weekday())
Weekday_Map=DataFrame([[x for x in range(0,8)],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]).transpose()
Weekday_Map.columns=['dday_num','dday']
Weekday_Map['dday_num']=Weekday_Map['dday_num'].astype('int')

DepartureDate_Map=DepartureDate_Map.merge(Weekday_Map,on='dday_num')

DepartureDate_Map=DepartureDate_Map[[x for x in DepartureDate_Map if x!='dday_num']]
DepartureDate_Map=DepartureDate_Map.sort_values(by='NewFormat')

Data=Data.merge(DepartureDate_Map,on=['DepDate'])
Data=Data[[x for x in Data.columns if x!='DepDate']]
Data=Data.rename(columns={'NewFormat': 'DepDate'})

#######################################


DepartureTime=Data['DepTime'].unique().tolist()
DepartureTime_Map=DataFrame([DepartureTime,[pd.to_datetime(x,format='%H:%M:%S') for x in DepartureTime]]).transpose()
DepartureTime_Map.columns=['DepTime','NewFormat']
DepartureTime_Map['dtime']=DepartureTime_Map['NewFormat'].apply(lambda x: (x-datetime.datetime.combine(x.date(),datetime.time(0,0))).seconds/60)

Data=Data.merge(DepartureTime_Map,on=['DepTime'])
Data=Data[[x for x in Data.columns if x!='DepTime']]
Data=Data.rename(columns={'NewFormat': 'DepTime'})

#######################################

Data['month']=Data['DepDate'].apply(lambda x: int(x.month))

#######################################

Data['Dprio']=(Data['DepDate']-Data['IssueDate']).apply(lambda x: float(x.days))
Data=Data.loc[(Data['Dprio']>0)&(Data['Dprio']<=366)&(Data['NumPax']>0)&(~pd.isnull(Data['DepTime'])),:]

Data=Data.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])

# CAPACITY SMOOTHING

#Data.loc[Data['Cabin Capacity']>70,'Cabin Capacity']=67

# Cabin Capacity

In [22]:
Capacity=Data[['DepDate','FltNum','dtime','Dprio','Cabin Capacity']].copy()

intermediate=Capacity.groupby(['DepDate','FltNum','dtime'])['Dprio'].min().reset_index().rename(columns={'Dprio': 'Dprio_min'})
intermediate['DepDate']=intermediate['DepDate'].apply(lambda x: pd.to_datetime(x))
Capacity['DepDate']=Capacity['DepDate'].apply(lambda x: pd.to_datetime(x))

Capacity_smooth=Capacity.merge(intermediate[[x for x in intermediate.columns if x!='Cabin Capacity']],
on=['DepDate','FltNum','dtime'])
Capacity_smooth=Capacity_smooth.loc[Capacity_smooth['Dprio']==Capacity_smooth['Dprio_min'],[x for x in Capacity_smooth.columns if 'Dprio' not in x]]

Dropdown_comb=Data[['DepDate','FltNum','dtime']].drop_duplicates()
Dropdown_comb.index=range(Dropdown_comb.shape[0])
Dprio_all=Data['Dprio'].unique().tolist()

Dropdown_comb=Dropdown_comb.loc[np.repeat(Dropdown_comb.index,len(Dprio_all)),:]
Dropdown_comb.index=range(Dropdown_comb.shape[0])

Dropdown_comb=pd.concat([Dropdown_comb,Series(Dprio_all*Dropdown_comb.drop_duplicates().shape[0])],axis=1)
Dropdown_comb=Dropdown_comb.rename(columns={0: 'Dprio'})

Dropdown_comb['DepDate']=Dropdown_comb['DepDate'].apply(lambda x: pd.to_datetime(x))
Capacity['DepDate']=Capacity['DepDate'].apply(lambda x: pd.to_datetime(x))

Dropdown_comb=Dropdown_comb.merge(Capacity,on=['DepDate','FltNum','dtime','Dprio'],how='left')
Dropdown_comb=Dropdown_comb.merge(Capacity_smooth.rename(columns={'Cabin Capacity': 'C_smooth'}),on=['DepDate','FltNum','dtime'])


Dropdown_comb['Cabin Capacity']=Dropdown_comb[['Cabin Capacity','C_smooth']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0],axis=1)

Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='C_smooth']]

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Dropdown_comb['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Capacity['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Dropdown_comb=Dropdown_comb.merge(Map_DepTime,on='dtime')
Dropdown_comb['dtime']=Dropdown_comb['deptime']
Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='deptime']]

# Pax Data

In [7]:
Pax=Data[['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday','NumPax']].copy()
Pax=Pax.set_index(['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday'])

Pax=Pax.groupby(level=[0,2,3,4,5,6,7]).diff().reset_index()
Pax=Pax.dropna(how='any')

Data_NoObs=Pax.loc[:,['DepDate','FltNum','yday','dtime','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['NumPax']=0

Data_NoObs=Data_NoObs[Pax.columns]

Pax=pd.concat([Pax,Data_NoObs])
Pax=Pax.drop_duplicates(subset=['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday'])

Pax=Pax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])
Pax.index=range(Pax.shape[0])

# Group bookings

In [8]:
GroupPax=Data[['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday','Group pax']].copy()
GroupPax=GroupPax.set_index(['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday'])

GroupPax=GroupPax.groupby(level=[0,2,3,4,5,6,7]).diff().reset_index()
GroupPax=GroupPax.dropna(how='any')
Data_NoObs=GroupPax.loc[:,['DepDate','FltNum','yday','dtime','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['Group pax']=0

Data_NoObs=Data_NoObs[GroupPax.columns]

GroupPax=pd.concat([GroupPax,Data_NoObs])
GroupPax=GroupPax.drop_duplicates(subset=['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday'])

GroupPax=GroupPax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])
GroupPax.index=range(GroupPax.shape[0])

GroupPax['Dprio']=-GroupPax['Dprio']
GroupPax['Group_pax_cumul']=GroupPax.groupby(['DepDate','FltNum','yday','dtime','Direction','month','dday'])['Group pax'].transform(lambda x: np.cumsum(x))
GroupPax['Dprio']=-GroupPax['Dprio']

# Remove group bookings from pax data and cabin capacity

In [9]:
# Pax=Pax.merge(GroupPax,on=['DepDate','Dprio','FltNum','yday','dtime','Direction','month','dday'])
# Pax['NumPax']=Pax['NumPax']-Pax['Group pax']
# Pax=Pax[[x for x in Pax.columns if x!='Group pax']]

##############################################################################

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in GroupPax['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Capacity['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

GroupPax=GroupPax.merge(Map_DepTime,on='dtime')
GroupPax['dtime']=GroupPax['deptime']
GroupPax=GroupPax[[x for x in GroupPax.columns if x!='deptime']]

# Dropdown_comb=Dropdown_comb.merge(GroupPax[['DepDate','FltNum','dtime','Dprio','Group_pax_cumul']],
# on=['DepDate','FltNum','dtime','Dprio'])
# Dropdown_comb['Cabin Capacity_removeGroup']=Dropdown_comb['Cabin Capacity']-Dropdown_comb['Group_pax_cumul']
# Dropdown_comb['Cabin Capacity_removeGroup']=Dropdown_comb[['Cabin Capacity_removeGroup', 'Cabin Capacity']].apply(lambda x: x[1] if x[0]>x[1] else x[0],axis=1)
# Dropdown_comb['Cabin Capacity']=Dropdown_comb['Cabin Capacity_removeGroup']
# Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='Group pax' and x!='Cabin Capacity_removeGroup']]

# Data export

### Pax

In [10]:
TrainingSet_Pax=Pax.loc[(Pax['DepDate']>=pd.to_datetime('2018-04-01'))\
&(Pax['DepDate']<pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))),].copy()
TestSet_Pax=Pax.loc[Pax['DepDate']>=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),].copy()

TrainingSet_Pax.to_csv('~/Data/Intermediate_Output/R_Training_Pax.csv',index=False)
TestSet_Pax['DepDate']=TestSet_Pax['DepDate'].apply(lambda x: pd.to_datetime(x))
TestSet_Pax.to_csv('~/Data/Intermediate_Output/R_Test_Pax.csv',index=False)

### Group Pax

In [11]:
GroupPax.to_csv('~/Data/Intermediate_Output/GroupPax.csv',index=False)

### Cabin Capacity

In [12]:
Dropdown_comb['Route']=route
Dropdown_comb.to_csv('~/Data/FrontEnd_Input/Capacity_forPlots_'+route+'.csv',index=False)

In [13]:
test=Pax.loc[(Pax['DepDate']>=pd.to_datetime('2019-05-01'))&(Pax['DepDate']<=pd.to_datetime('2019-05-31')),:]